## 用C/C++构建命令行工具

`unistd.h`是C语言的一个准标准库,它是linux,mac osx等类unix系统的自带的库,但windows并不支持.要在windows下使用需要自己写一个头文件,如下:

```cpp
/** This file is part of the Mingw32 package. 
*  unistd.h maps     (roughly) to io.h 
*/  
#ifndef _UNISTD_H  
#define _UNISTD_H  
#include <io.h>  
#include <process.h>  
#endif /* _UNISTD_H */ 

```


使用它便利的构建命令行工具`getopt`

`getopt()`用来分析命令行参数.参数`argc`和`argv`是由`main()`传递的参数个数和内容.参数`optstring`则代表欲处理的选项字符串.此函数会返回在`argv`中下一个的选项字母,此字母会对应参数`optstring`中的字母.如果选项字符串里的字母后接着冒号":".则表示还有相关的参数,全域变量`optarg` 即会指向此额外参数.如果`getopt()`找不到符合的参数则会印出错信息,并将全域变量`optopt`设为"?"字符,如果不希望`getopt()`印出错信息,则只要将全域变量`opterr`设为0即可.

### 短参数的定义

`getopt()`使用`optstring`所指的字串作为短参数列表,像`"1ac:d::"`就是一个短参数列表.短参数的定义是一个`'-'`后面跟一个字母或数字,象`-a`, `-b`就是一个短参数.每个数字或字母定义一个参数.

其中短参数在`getopt`定义里分为三种：

+ 不带值的参数,它的定义即是参数本身
+ 必须带值的参数,它的定义是在参数本身后面再加一个冒号
+ 可选值的参数,它的定义是在参数本身后面加两个冒号

在这里拿上面的`"1ac:d::"`作为样例进行说明,其中的
+ `1`,`a`就是不带值的参数
+ `c`是必须带值的参数
+ `d`是可选值的参数

在实际调用中，`'-1 -a -c cvalue -d'`, `'-1 -a -c cvalue -ddvalue'`, `'-1a -ddvalue -c cvalue'`都是合法的.
这里需要注意三点:

+ 不带值的参数可以连写,像`1`和`a`是不带值的参数,它们可以`-1` `-a`分开写,也可以`-1a`或`-a1`连写.
+ 参数不分先后顺序,`'-1a -c cvalue -ddvalue'`和`'-d -c cvalue -a1'`的解析结果是一样的.
+ 要注意可选值的参数的值与参数之间不能有空格,必须写成`-ddvalue`这样的格式,如果写成`-d dvalue`这样的格式就会解析错误.

### 返回值

`getopt()`每次调用会逐次返回命令行传入的参数. 　 
+ 当没有参数的最后的一次调用时,`getopt()`将返回-1.　 
+ 当解析到一个不在`optstring`里面的参数，或者一个必选值参数不带值时,返回'?'.　 
+ 当`optstring`是以':'开头时,缺值参数的情况下会返回':',而不是'?'.

In [1]:
%%writefile src/C10/sqrt_opt.cpp
#include <stdlib.h>
#include <unistd.h>
#include <iostream>
#include <string>
#include <math.h>
using std::string;
using std::cout;
using std::endl;

int main(int argc, char **argv)  {  
    int ch; 
    double result;
    opterr = 0;  
    string version = "0.1.0";
    while ((ch = getopt(argc,argv,"s:v\n"))!=-1){  
        switch(ch){  
            case 'v':  
                cout<<("option v: %s\n",version)<<endl;  
                break;  
            case 's':
                result = sqrt(atoi(optarg));
                cout<<("%f\n",result)<<endl;
                break;
            default:  
                cout<<"wrong args\n"<<endl;  
        }  
    }  
    return 0;
}  

Writing src/C10/sqrt_opt.cpp


In [2]:
!g++-7 -o bin/sqrt_opt src/C10/sqrt_opt.cpp

In [3]:
!./bin/sqrt_opt -v

0.1.0


In [4]:
!./bin/sqrt_opt -s36

6
